**IOC 8 – Covert DNS Beaconing via Google Resolver**

**1. Attacker Analogy**

A teenager in a tightly monitored house isn’t allowed to make phone calls past 10:00 p.m. The only approved line is the family landline, and all outgoing calls are logged and visible. But this kid has found a prepaid burner phone hidden in their desk. Every night, like clockwork, they step into the garage and tap the same number—just one ring. No conversation. No noise. Then they return to bed.

Now imagine a workstation doing the same thing. Every 30 minutes, it sends a DNS query to Google’s public DNS server (8.8.8.8)—bypassing internal DNS, querying strange subdomains, and getting no response. The system is quiet, no user is active, and nothing appears suspicious—unless you’re watching the pattern.

**2. Source of IOC (Telemetry Origin)**
This IOC originated from NetFlow telemetry analysis. Host 192.168.12.21 was observed sending UDP port 53 traffic to external IP 8.8.8.8 on a precise 30-minute interval. No internal DNS server logs contained these domain requests, indicating direct resolver bypass.

Softened Interpretation (with Parenthetical Mapping):
We noticed the host was calling someone, but not through the house line. (Parenthetical: NetFlow logs showed repeated DNS queries to 8.8.8.8, bypassing internal DNS servers.) The timing wasn’t random—it was like a scheduled call every half hour. (NetFlow volume/time analysis flagged the fixed interval.) And nobody seemed to be home when the calls were made. (No corresponding user logon events.)

**3. Triage Framework Declaration**
Triage Type: Network Activity Indicator Triage
Protocol Used: Network Triage Protocol
Tools Used (Required Order):

**Nmap Scan**

Targeted host 192.168.12.21

Confirmed UDP/53 egress was allowed, no internal DNS service present

No unexpected listening services detected

**Windows Event Logs**

Event ID 5156: Showed svchost.exe repeatedly permitted to initiate outbound UDP/53 traffic

Event ID 5145: Reviewed, no SMB artifacts related to IOC

Event ID 4672: No high-privilege assignments detected

**NetFlow Analysis**

Showed stateless, unidirectional DNS queries to 8.8.8.8 at regular 30-minute intervals. NetFlow showed repeated outbound UDP/53 packets from 192.168.12.21 to 8.8.8.8, with no return traffic, confirming the communication was stateless—used for signaling, not resolution.

Queried domains were randomized (e.g., a7r29s.dnsbeacon.tk), indicating algorithmic generation

Softened Interpretation (with Parenthetical Mapping):
We scanned the machine to see what doors it might have open. (Nmap confirmed outbound UDP/53 was permitted; no DNS server process or unexpected listeners found.)

Then we checked its permissions—was it allowed to use that door? (Windows Event ID 5156 showed svchost.exe outbound connections repeatedly permitted.)

Finally, we looked at its call history—the pattern of communication. (NetFlow confirmed regular outbound DNS traffic to 8.8.8.8 with randomized domains.) The host wasn’t yelling—it was whispering, over and over, at the same time, like a digital metronome.

**4. OS Layer with Key Behaviors**
Mapped Host OS Layer: Layer 6 – Network Communication Context

Function: Manages outbound connections using DNS, HTTPS, SMB, etc.

Observation: svchost.exe issued repeated DNS queries, externally routed, with no user trigger

Softened Interpretation (with Parenthetical Mapping):
Inside the system, Layer 6 handles the talking—when and how to speak to the outside world. (This activity was driven by svchost.exe using Layer 6 functions to send DNS queries.)

This wasn’t a user typing something. It was a process set to murmur at regular intervals. (Automated, repeating DNS queries, likely from a script or task scheduler.)

**5. Cross-Layer Interaction Pivots**
Pivot Initiated: From Network Triage Protocol → to Host-Based Local Triage Protocol

Justification:

DNS behavior originated from an internal process (svchost.exe)

No user activity during beaconing periods

Timing implies scheduled task, registry autorun, or implant activity

Protocol Invoked:
Host-Based Local Triage Protocol
Tools Used in Proper Order:

**Windows Event Logs**

Cross-reference Event ID 4688: Check if a process initiated DNS behavior

**EDR Telemetry (if available)**

Inspect parent-child process relationships, command-line arguments

**File System and Registry Inspection**

Focus on HKCU\Run, Task Scheduler, unusual binaries in %APPDATA%. 

When triaging a suspicious host, we focus on areas like HKCU\Run (Current User autoruns), Task Scheduler entries, and binaries in %APPDATA%, because these are prime real estate for persistence mechanisms. Attackers often want their implant to survive a reboot and run silently in the background without raising alarms. 

The HKCU\Run registry key is a classic method to achieve this—it instructs Windows to launch a specified binary every time the user logs in, and it doesn’t require elevated privileges, making it ideal for stealthy persistence. 

Similarly, Task Scheduler allows attackers to create recurring tasks (e.g., every 30 minutes) that can execute scripts or binaries invisibly. 

The %APPDATA% directory, part of the user profile, is a common drop location because it is writable by the user, rarely monitored, and often excluded from routine antivirus scanning. 

In combination, these three areas can sustain a low-noise, repeating behavior—such as DNS beaconing—without tripping signature-based defenses. We inspect them to find the implant’s trigger, its executable payload, and the scheduler or startup mechanism that brings it to life.

**Volatile Memory Capture**

Analyze memory of svchost.exe using Volatility (malfind, ldrmodules, netscan)

Softened Interpretation (with Parenthetical Mapping):
We heard the whisper—now we need to find the source of the voice. We check the alarm clock (Windows Scheduled Tasks) to see if it was set to ring every 30 minutes. We look for notes taped to the wall (registry autorun keys). We review the memory of the speaker (volatile memory capture for injected DLLs in svchost.exe). Everything points to something inside the house—set on a timer.

**6. OSI Layer Relevance**
Layer 3 – Network Layer: IP routing to 8.8.8.8

Layer 4 – Transport Layer: UDP connection, stateless

Layer 7 – Application Layer: DNS protocol behavior, randomized queries

Softened Interpretation (with Parenthetical Mapping):
The host used the whole highway system to get its signal out. (Layer 3 handled destination IP; Layer 4 carried it via UDP; Layer 7 wrapped the message in DNS.) The queries weren’t just odd—they were structured to hide in plain sight, like someone whispering in the middle of a parade.

**7. Attacker Behavior Interpretation**
Technique: Timed DNS beaconing to external resolver

Method: Uses svchost.exe for legitimacy, randomized subdomains, fixed interval signaling

Purpose: Low-noise Command-and-Control (C2) check-in or staging beacon

MITRE Technique: T1071.004 – Application Layer Protocol: DNS

Softened Interpretation (with Parenthetical Mapping):
This wasn’t a smash-and-grab. This was a quiet ping, checking if the line was still open. (30-minute interval beaconing using DNS queries via svchost.exe.) The attacker wasn’t ready to act yet—but they were watching, waiting, and keeping the backchannel warm.

**8. Defender Action Summary**
Isolate host 192.168.12.21 from the network

Apply Host Triage Protocol to analyze persistence and execution triggers

Block external DNS—restrict outbound UDP/53 to internal resolvers only

Sinkhole domains associated with observed beacon patterns

Correlate MITRE Technique (T1071.004) in SIEM for similar host behaviors

Flag svchost.exe anomalies in process telemetry

Softened Interpretation (with Parenthetical Mapping):
We unplugged the line, then went room to room, looking for evidence of how the calls were being placed. (Network isolation, host inspection of startup infrastructure and memory.) We blocked the number being called (8.8.8.8) and made sure no other phones in the house were hiding under the bed.

**9. Attacker Strategy Notes**
Strategic patience over immediate exploitation

Use of legitimate infrastructure (Google DNS) to avoid blacklist detection

Likely tied to modular implant awaiting C2 activation

Strong evasion: low volume, non-payload, protocol blending

Softened Interpretation (with Parenthetical Mapping):
This wasn’t theft—it was a ping, a silent touch on the doorknob. (Automated DNS signal without payloads or responses.) The attacker left a line open—not to act now, but to come back later. And they did it in a way that most defenses wouldn’t even notice.

**10. Final Analyst Reflection and Softened Narrative**
This case underscores why pattern recognition matters. There was no malware alert, no virus detected, no user complaint. Just a beat—regular, subtle, quiet.

Using the correct protocols, we broke this beat down. The Network Triage Protocol exposed the signal. The Host Triage Protocol showed where it started. And our layer mapping confirmed that a legitimate process (svchost.exe) was doing something very illegitimate.



**11. Who’s Who – Object Role Clarification**
Object	                 Role in IOC
192.168.12.21	Internal host emitting DNS queries
8.8.8.8	Google Public DNS, abused as external resolver
UDP port 53	Transport layer used for beaconing
svchost.exe	Legitimate Windows process, possibly hijacked
Nmap	Confirmed no listener, outbound DNS allowed
Event ID 5156	Verified network connection from svchost.exe allowed
NetFlow	Confirmed exact 30-minute intervals and randomized DNS queries
Volatility	Memory forensics tool/ svchost.exe examined/ no positive findings noted
exampledomain.tk	Suspicious domain queried during beaconing behavior

